In [ ]:
using ArgParse: ArgParseSettings, @add_arg_table!, parse_args
using Statistics: mean
using Printf
using Knet

include("../latentplan/LPCore.jl")
include("../latentplan/setup.jl")
using .LPCore

In [ ]:
losssum(prediction) = mean(prediction[2] + prediction[3] + prediction[4])

function vq_train(config, model::VQContinuousVAE, dataset; n_epochs=1, log_freq=100)
    # set optimizers
    opt_decay = AdamW(lr=config["learning_rate"], beta1=config["betas"][1], beta2=config["betas"][2], weight_decay=config["weight_decay"], gclip=config["grad_norm_clip"])
    opt_no_decay = AdamW(lr=config["learning_rate"], beta1=config["betas"][1], beta2=config["betas"][2], weight_decay=0.0, gclip=config["grad_norm_clip"])

    for p in paramlist_decay(model)
        p.opt = clone(opt_decay)
    end
    for p in paramlist_no_decay(model)
        p.opt = clone(opt_no_decay)
    end

    n_tokens = 0
    loader = DataLoader(dataset; shuffle=true, batch_size=config["batch_size"])

    for epoch in 1:n_epochs
        losses = []
        for (it, batch) in enumerate(loader)
            y = batch[end-1]
            n_tokens += cumprod(size(y))

            if n_tokens < config["warmup_tokens"]
                # linear warmup
                lr_mult = float(n_tokens) / float(max(1, config["warmup_tokens"]))
            else
                # cosine learning rate decay
                progress = float(n_tokens - config["warmup_tokens"]) / float(
                    max(1, config["final_tokens"] - config["warmup_tokens"])
                )
                lr_mult = max(0.1, 0.5 * (1.0 + cos(pi * progress)))
            end

            if config["lr_decay"]
                lr = config["learning_rate"] * lr_mult
                # TODO: param_group learning rate
                for p in paramlist(model)
                    p.opt.lr = lr
                end
            else
                lr = config["learning_rate"]
            end

            # forward the model
            total_loss = @diff losssum(model(batch...))
            push!(losses, value(total_loss))
            for p in paramlist(model)
                update!(p, grad(total_loss, p))
            end
        end
    end
end

In [ ]:
s = ArgParseSettings()
@add_arg_table! s begin
    "--dataset"
        help = "which environment to use"
        arg_type = String
        default = "halfcheetah-medium-expert-v2"
    "--exp_name"
        help = "name of the experiment"
        arg_type = String
        default = "debug"
    "--seed"
        help = "seed"
        arg_type = Int
        default = 42
    "--config"
        help = "relative jl file path with configurations"
        arg_type = String
        default = "../config/vqvae.jl"
end

#######################
######## setup ########
#######################

super_args = parse_args([], s)
args = parser(super_args, experiment="train");

# Dataset

In [ ]:
env_name = occursin("-v", args["dataset"]) ? args["dataset"] : args["dataset"] * "-v0"

# env params
sequence_length = args["subsampled_sequence_length"] * args["step"]
args["logbase"] = expanduser(args["logbase"])
args["savepath"] = expanduser(args["savepath"])
if !isdir(args["savepath"])
    mkpath(args["savepath"])
end

dataset = SequenceDataset(
    env_name;
    penalty=args["termination_penalty"], 
    sequence_length=sequence_length, 
    step=args["step"], 
    discount=args["discount"], 
    disable_goal=args["disable_goal"], 
    normalize_raw=args["normalize"], 
    normalize_reward=args["normalize_reward"],
    max_path_length=args["max_path_length"],
)

obs_dim = dataset.observation_dim
act_dim = dataset.action_dim
if args["task_type"] == "locomotion"
    transition_dim = obs_dim+act_dim+3
else
    transition_dim = 128+act_dim+3
end

block_size = args["subsampled_sequence_length"] * transition_dim # total number of dimensionalities for a maximum length sequence (T)

print(
    "Dataset size: $(length(dataset)) |
    Joined dim: $transition_dim
    observation: $obs_dim, action: $act_dim | Block size: $block_size"
)

# Model

In [1]:
model_config = deepcopy(args)
model_config["block_size"] = block_size
model_config["observation_dim"] = obs_dim
model_config["action_dim"] = act_dim
model_config["transition_dim"] = transition_dim
model_config["n_embd"] = args["n_embd"] * args["n_head"]
model_config["vocab_size"] = args["N"]

model = VQContinuousVAE(model_config);
model.padding_vector = normalize_joined_single(dataset, zeros(Float32, model.transition_dim-1))


LoadError: UndefVarError: args not defined

## Model Params

In [ ]:
# component = model.model.ln_f
# num_params = 0
# for (i, param) in enumerate(paramlist(model))
#     println(i, " - ", length(param))
#     num_params += length(param)
# end
# print("Number of parameters in model: ", num_params)

# Train process

In [ ]:
loader = DataLoader(dataset; shuffle=false, batch_size=args["batch_size"])
batch = nothing
for (it, b) in enumerate(loader)
    batch = b
    break
end

In [ ]:
joined_inputs, targets, mask, terminals = batch;

In [ ]:
joined_dimension, t, b = size(joined_inputs)

In [ ]:
model.padding_vector = normalize_joined_single(dataset, zeros(Float32, model.transition_dim-1));

In [ ]:
padded = repeat(model.padding_vector, 1, t, b);

In [ ]:
terminal_mask = repeat(deepcopy(.~terminals), size(joined_inputs, 1), 1, 1);

In [ ]:
joined_inputs = joined_inputs .* terminal_mask .+ padded .* (1 .- terminal_mask);

In [ ]:
state = joined_inputs[1:model.observation_dim, 1, :]

In [ ]:
cat(joined_inputs, terminals, dims=1)